In [ ]:
import datetime
import mysql.connector

# Connect to MySQL database
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="tasianna",
    database="InstagramDB"
)

# a cursor that returns results as dictionaries (column: value)
cursor = cnx.cursor(dictionary=True)




Queries showing how to display information from tables, calling procedures etc 




In [102]:
# Query 1 Display each post with its author’s username.
# Host Variable - type_of_content
# setting host variable
type_of_content = "Photo"
#assigning code to variable
query1 = """
SELECT p.PostID, u.Username AS Author, p.Content, p.Date, p.TypeOfContent
FROM Post p
JOIN User u ON p.UserID = u.UserID
WHERE (%s IS NULL OR p.TypeOfContent = %s)
ORDER BY p.Date;
"""
#using cursor execute to run commands and fetch to retrieve
cursor.execute(query1, (type_of_content, type_of_content))
results = cursor.fetchall()
#using for loop to print each one
print("Query 1 test results\n")
print("Posts and their authors:\n")
for row in results:
    print(row)


Query 1 test results

Posts and their authors:

{'PostID': 1, 'Author': 'tasianna', 'Content': 'Morning Run (edited caption)', 'Date': datetime.date(2025, 9, 20), 'TypeOfContent': 'Photo'}
{'PostID': 3, 'Author': 'matthew', 'Content': 'Best Gelato Ever', 'Date': datetime.date(2025, 9, 22), 'TypeOfContent': 'Photo'}


In [103]:
# Query 2 - Print comments, commenter username, and the text of the post they commented on.
# Host Variable - post_id
post_id = 1
#assigning query code to variable
query2 = """
SELECT c.CommentID, u.Username AS Commenter, c.Content AS CommentText,
       p.Content AS PostText, c.Date
FROM Comment c
JOIN User u ON c.UserID = u.UserID
JOIN Post p ON c.PostID = p.PostID
WHERE p.PostID = %s
ORDER BY c.Date;
"""
#using cursor execute to run commands and fetch to retrieve
cursor.execute(query2, (post_id,))
results = cursor.fetchall()
#using for loop to print each one
print("Query 2 test results\n")
print(f"Comments for Post {post_id}:\n")
for row in results:
    print(row)


Query 2 test results

Comments for Post 1:

{'CommentID': 1, 'Commenter': 'michael22', 'CommentText': 'You look great!', 'PostText': 'Morning Run (edited caption)', 'Date': datetime.date(2025, 9, 21)}
{'CommentID': 2, 'Commenter': 'matthew', 'CommentText': 'Awesome job!', 'PostText': 'Morning Run (edited caption)', 'Date': datetime.date(2025, 9, 21)}


In [104]:
# Query 3 Find users with more than one follower.
# Host Variables - min followers and follower status
follower_status = "Accepted"
min_followers = 1
# using count
query3 = """
WITH FollowerCounts AS (
    SELECT FollowedID, COUNT(*) AS TotalFollowers
    FROM Follower
    WHERE Status = %s
    GROUP BY FollowedID
)
SELECT u.Username, fc.TotalFollowers
FROM FollowerCounts fc
JOIN User u ON u.UserID = fc.FollowedID
WHERE fc.TotalFollowers > %s
ORDER BY fc.TotalFollowers DESC;
"""
# ^ used %s as placeholder and ordered by descending order
# execute and fetch to run and retrieve
cursor.execute(query3, (follower_status, min_followers))
results = cursor.fetchall()
# for loop to print

print("Query 3 test results\n")
print("Users with more than", min_followers, "followers:\n")
for row in results:
    print(row)


Query 3 test results

Users with more than 1 followers:

{'Username': 'tasianna', 'TotalFollowers': 2}


In [105]:
# Query 4: Find Amount of Posts for each user
# Host Variable- since_date
since_date = datetime.date(2025, 9, 20)
# counting post id for each username present, grouping by username and ordering in descending order
query4 = """
SELECT u.Username, COUNT(p.PostID) AS PostCount
FROM User u
LEFT JOIN Post p ON p.UserID = u.UserID AND p.Date >= %s
GROUP BY u.Username
ORDER BY PostCount DESC;
"""
# execute and fetch to run and retrieve
cursor.execute(query4, (since_date,))
results = cursor.fetchall()
print("Query 4 test results\n")
print(f"Post counts since {since_date}:\n")
for row in results:
    print(row)


Query 4 test results

Post counts since 2025-09-20:

{'Username': 'matthew', 'PostCount': 1}
{'Username': 'michael22', 'PostCount': 1}
{'Username': 'tasianna', 'PostCount': 1}
{'Username': 'nadia', 'PostCount': 0}


In [106]:
# Query 5: Count total likes per post
# Host variables- start_date, end_date
# assigning the start and end date to plug in later
start_date = datetime.date(2025, 9, 20)
end_date = datetime.date(2025, 9, 23)
# count to count each likeid and putting in a date range to checl
query5 = """
SELECT p.PostID, p.Content, COUNT(l.LikeID) AS TotalLikes
FROM Post p
LEFT JOIN Likes l ON l.PostID = p.PostID AND l.Date BETWEEN %s AND %s
GROUP BY p.PostID, p.Content
ORDER BY TotalLikes DESC;
"""

cursor.execute(query5, (start_date, end_date))
results = cursor.fetchall()

print("Query 5 test results\n")
print(f"Total likes per post from {start_date} to {end_date}:\n")
for row in results:
    print(row)


Query 5 test results

Total likes per post from 2025-09-20 to 2025-09-23:

{'PostID': 1, 'Content': 'Morning Run (edited caption)', 'TotalLikes': 2}
{'PostID': 2, 'Content': 'My first 5K!', 'TotalLikes': 1}
{'PostID': 3, 'Content': 'Best Gelato Ever', 'TotalLikes': 1}


In [107]:
# Query 6 : See posts liked by a specific user
# Host Variable liked_by_username

liked_by_username = "michael22"
# see if a post if username if present within certian post id
query6 = """
SELECT p.PostID, p.Content, p.Date
FROM Post p
WHERE p.PostID IN (
    SELECT l.PostID
    FROM Likes l
    WHERE l.UserID = (SELECT UserID FROM User WHERE Username = %s)
)
ORDER BY p.Date;
"""
# ordering by date with datetime
# execute and fetch to execute and retrieve
cursor.execute(query6, (liked_by_username,))
results = cursor.fetchall()

print("Query 6 test results\n")
print(f"Posts liked by {liked_by_username}:\n")
for row in results:
    print(row)


Query 6 test results

Posts liked by michael22:

{'PostID': 1, 'Content': 'Morning Run (edited caption)', 'Date': datetime.date(2025, 9, 20)}


2. Stored routine implementation:
1) (3 pts) Define a procedure that uses an aggregate function.


In [108]:

# calling stored procedure
print("\nCalling procedure: CountPostsByUser(1)")
cursor.callproc("CountPostsByUser", [1])

for result in cursor.stored_results():
    rows = result.fetchall()
    for row in rows:
        # row is a dict
        print(row)

# 2)	(3 pts) Define a function that returns a value.
# calling stored function
print("\nCalling function: GetLikesForPost(1)")
cursor.execute("SELECT GetLikesForPost(%s) AS total_likes", (1,))
row = cursor.fetchone()          # dict like {'total_likes': 2}
likes = row["total_likes"]
print("Total likes for Post 1:", likes)







Calling procedure: CountPostsByUser(1)
{'Username': 'tasianna', 'TotalPosts': 1}

Calling function: GetLikesForPost(1)
Total likes for Post 1: 2


/var/folders/28/q8cy1x493vd1kftn2zrjtlvw0000gn/T/ipykernel_47370/2068312437.py:5: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


In [109]:
import datetime
import mysql.connector

def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="tasianna",
        database="InstagramDB"
    )



3. (10*3=30 pts) Company task implementations.
Assume THREE business tasks for your company and implement them


In [120]:
# Function 1.
# I want to create a function that will run an ad campaign on a certian date. Advertiser will need who it is targeting (what UserID)
import mysql.connector
from datetime import date

def create_campaign(advertiser_name, content, content_type, start, end):
    cnx = mysql.connector.connect(host="localhost", user="root", password="tasianna", database="InstagramDB")
    cur = cnx.cursor()
# selecting user and executing campaign
    sql = """
    INSERT INTO Advertisement (AdvertiserID, Content, ContentType, StartDate, EndDate)
    VALUES ((SELECT UserID FROM User WHERE Username=%s), %s, %s, %s, %s);
    """
    cur.execute(sql, (advertiser_name, content, content_type, start, end))
    cnx.commit()

    print(f"Campaign created for {advertiser_name}: '{content}' ({content_type})")

    cur.close()
    cnx.close()

#testing
create_campaign("tasianna", "Handmade Bracelet Sale 50% off!", "Photo", date(2025, 10, 25), date(2025, 11, 25))


Campaign created for tasianna: 'Handmade Bracelet Sale 50% off!' (Photo)


In [121]:
#Function 2
# the advertiser should be able to pull a report on thier ad campaign that shows how many likes it got
def likes_summary_report(advertiser_name):
    cnx = mysql.connector.connect(host="localhost", user="root", password="tasianna", database="InstagramDB")
    cur = cnx.cursor(dictionary=True)
#using left join on postid and using %s as a placeholder
    sql = """
    SELECT u.Username, p.PostID, COUNT(l.LikeID) AS TotalLikes
    FROM Post p
    JOIN User u ON u.UserID = p.UserID
    LEFT JOIN Likes l ON l.PostID = p.PostID
    WHERE u.Username = %s
    GROUP BY p.PostID, u.Username;
    """
    cur.execute(sql, (advertiser_name,))
#printing a summary report in a nice way
    results = cur.fetchall()
    print(f"\nLike Summary Report for {advertiser_name}'s Campaign:")
    for r in results:
        print(f"Post {r['PostID']} → {r['TotalLikes']} likes")

    cur.close()
    cnx.close()

# testrun
likes_summary_report("tasianna")



Like Summary Report for tasianna's Campaign:
Post 1 → 2 likes


In [125]:
# Function 3
# Advertiser should have an easy way of seeing how much they have spent on campaign so far
# In this example I am making it 25 cents per click
#definig function
def spent_report(username):
    cnx = mysql.connector.connect(host="localhost", user="root", password="tasianna", database="InstagramDB")
    cur = cnx.cursor(dictionary=True)
#creating likes count and left joining on post id, grouping by username
    sql = """
    SELECT u.Username, COUNT(l.LikeID) AS LikesCount
    FROM User u
    JOIN Post p ON u.UserID = p.UserID
    LEFT JOIN Likes l ON p.PostID = l.PostID
    WHERE u.Username = %s
    GROUP BY u.Username;
    """
    cur.execute(sql, (username,))
    result = cur.fetchone()
# soldifying variables and printing results
    likes = result["LikesCount"]
    PricePerClick = likes * 0.25
    print(f"\n{username} has {likes} likes on the Ad Campaign= ${PricePerClick:.2f} total spent, with .25 per click.")

    cur.close()
    cnx.close()

# Example run
spent_report("tasianna")



tasianna has 2 likes on the Ad Campaign= $0.50 total spent, with .25 per click.
